In [1]:
import jax
import jax.numpy as jnp
from leaguedata.util import find_streaks
from jax.tree_util import tree_flatten, tree_map
from leaguedata.data import get_history_dict

history = get_history_dict()
history_array = jnp.asarray(tree_flatten(tree_map(lambda x : x[-85:], history))[0])
true_win_streaks_counts, true_loss_streaks_counts = jax.vmap(find_streaks)(history_array)
true_win_streaks_freq = true_win_streaks_counts.sum(axis=0) / true_win_streaks_counts.sum()
true_loss_streaks_freq = true_loss_streaks_counts.sum(axis=0) / true_loss_streaks_counts.sum()

In [2]:
from jax.random import split, PRNGKey
from tqdm.notebook import trange
from leaguedata.model import generate_simulation_from_best_fit, generate_nasty_loser_q, generate_obvious_loser_q

mock_win_streaks_counts, mock_loss_streaks_counts = [], []
loserq_win_streaks_counts, loserq_loss_streaks_counts = [], []

for i, key in zip(trange(100), split(PRNGKey(42), 100)):

    key1, key2 = split(key, 2)
    
    simulated_model = generate_simulation_from_best_fit(number_of_players=3100, key=key1)
    res = jax.vmap(find_streaks)(simulated_model)
    mock_win_streaks_counts.append(res[0])
    mock_loss_streaks_counts.append(res[1])    

    simulated_loserq = generate_obvious_loser_q(number_of_players=3100, key=key2)
    res = jax.vmap(find_streaks)(simulated_loserq)
    loserq_win_streaks_counts.append(res[0])
    loserq_loss_streaks_counts.append(res[1])    

mock_win_streaks_counts = jnp.asarray(mock_win_streaks_counts)
mock_loss_streaks_counts = jnp.asarray(mock_loss_streaks_counts)
loserq_win_streaks_counts = jnp.asarray(loserq_win_streaks_counts)
loserq_loss_streaks_counts = jnp.asarray(loserq_loss_streaks_counts)

  0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
mock_win_streaks_freq = mock_win_streaks_counts.sum(axis=1) / mock_win_streaks_counts.sum(axis=(1, 2))[:, None]
mock_loss_streaks_freq = mock_loss_streaks_counts.sum(axis=1) / mock_loss_streaks_counts.sum(axis=(1, 2))[:, None]
loserq_win_streaks_freq = loserq_win_streaks_counts.sum(axis=1) / loserq_win_streaks_counts.sum(axis=(1, 2))[:, None]
loserq_loss_streaks_freq = loserq_loss_streaks_counts.sum(axis=1) / loserq_loss_streaks_counts.sum(axis=(1, 2))[:, None]

In [9]:
import numpy as np 
import plotly.graph_objects as go

fig = go.Figure()

bins = dict( # bins used for histogram
        start=1,
        end=30,
        size=1
    )


fig.add_trace(
    go.Bar(
        x=np.arange(1, len(true_win_streaks_freq), 1),
        y=true_win_streaks_freq,
        name='True win streaks',
        marker_color='rgba(60, 179, 113, 0.6)',
    )
)

fig.add_trace(
    go.Bar(
        x=np.arange(1, len(true_loss_streaks_freq), 1),
        y=true_loss_streaks_freq,
        name='True loss streaks',
        marker_color='rgba(199, 21, 133, 0.6)',
    )
)

fig.update_xaxes(
    tickmode='linear', 
    tick0=1, 
    dtick=1, 
    range=[0.5, 18.],
    showgrid=True, 
    title='',
    minor=dict(showgrid=True)
)

fig.update_yaxes(
    type='log', 
    title='Probability', 
    range=[-5, 0], 
    showgrid=True, 
    minor=dict(showgrid=True)
)

fig.update_layout(
    hovermode="x",
    margin=dict(l=20, r=20, t=20, b=20),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    width=600, height=300,  
)

fig.show()

with open("docs/loserQ/assets/history_streak_histogram.json", "w") as f:
    f.write(fig.to_json())

In [10]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=np.arange(1, 20, 1),
        y=true_win_streaks_freq,
        name='True win streaks',
        marker_color='rgba(60, 179, 113, 0.6)',
    )
)

fig.add_trace(
    go.Scatter(
        x=np.arange(1, 20, 1),
        y=np.mean(mock_win_streaks_freq, axis=0) - 2*np.std(mock_win_streaks_freq, axis=0),
        line=dict(color='rgba(60, 179, 113, 0.9)'),
        name='Expected values',
        showlegend=False,
        legendgroup='fake_lose'
    )
)

fig.add_trace(
    go.Scatter(
        x=np.arange(1, 20, 1),
        y=np.mean(mock_win_streaks_freq, axis=0) + 2*np.std(mock_win_streaks_freq, axis=0),
        line=dict(color='rgba(60, 179, 113, 0.9)'),
        fillcolor='rgba(60, 179, 113, 0.6)',
        fill='tonexty',
        name='Expected values',
        legendgroup='fake_lose'
    )
)

fig.add_trace(
    go.Scatter(
        x=np.arange(1, 20, 1),
        y=np.mean(loserq_win_streaks_freq, axis=0) - 2*np.std(loserq_win_streaks_freq, axis=0),
        line=dict(color='rgba(255, 191, 0, 0.9)'),
        name='Expected values',
        showlegend=False,
        legendgroup='fake_loserq'
    )
)

fig.add_trace(
    go.Scatter(
        x=np.arange(1, 20, 1),
        y=np.mean(loserq_win_streaks_freq, axis=0) + 2*np.std(loserq_win_streaks_freq, axis=0),
        line=dict(color='rgba(255, 191, 0, 0.9)'),
        fillcolor='rgba(255, 191, 0, 0.6)',
        fill='tonexty',
        name='Mock LoserQ',
        legendgroup='fake_loserq'
    )
)

fig.update_xaxes(
    tickmode='linear', 
    tick0=1, 
    dtick=1, 
    range=[0.5, 18.],
    showgrid=True, 
    title='',
    minor=dict(showgrid=True)
)

fig.update_yaxes(
    type='log', 
    title='Probability', 
    range=[-5, 0], 
    showgrid=True, 
    minor=dict(showgrid=True)
)

fig.update_layout(
    hovermode="x",
    margin=dict(l=20, r=20, t=20, b=20),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    width=600, height=300,  
)

# Save to JSON file
with open("docs/loserQ/assets/mock_streak_histogram_win.json", "w") as f:
    f.write(fig.to_json(pretty=True))
    
fig.show()

In [11]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=np.arange(1, 20, 1),
        y=true_loss_streaks_freq,
        name='True loss streaks',
        marker_color='rgba(199, 21, 133, 0.6)',
    )
)

fig.add_trace(
    go.Scatter(
        x=np.arange(1, 20, 1),
        y=np.mean(mock_loss_streaks_freq, axis=0) - 2*np.std(mock_loss_streaks_freq, axis=0),
        line=dict(color='rgba(199, 21, 133, 0.9)'),
        name='Expected values',
        showlegend=False,
        legendgroup='fake_lose'
    )
)

fig.add_trace(
    go.Scatter(
        x=np.arange(1, 20, 1),
        y=np.mean(mock_loss_streaks_freq, axis=0) + 2*np.std(mock_loss_streaks_freq, axis=0),
        line=dict(color='rgba(199, 21, 133, 0.9)'),
        fillcolor='rgba(199, 21, 133, 0.6)',
        fill='tonexty',
        name='Expected values',
        legendgroup='fake_lose'
    )
)

fig.add_trace(
    go.Scatter(
        x=np.arange(1, 20, 1),
        y=np.mean(loserq_loss_streaks_freq, axis=0) - 2*np.std(loserq_loss_streaks_freq, axis=0),
        line=dict(color='rgba(255, 191, 0, 0.9)'),
        name='Expected values',
        showlegend=False,
        legendgroup='fake_loserq'
    )
)

fig.add_trace(
    go.Scatter(
        x=np.arange(1, 20, 1),
        y=np.mean(loserq_loss_streaks_freq, axis=0) + 2*np.std(loserq_loss_streaks_freq, axis=0),
        line=dict(color='rgba(255, 191, 0, 0.9)'),
        fillcolor='rgba(255, 191, 0, 0.6)',
        fill='tonexty',
        name='Mock LoserQ',
        legendgroup='fake_loserq'
    )
)

fig.update_xaxes(
    tickmode='linear', 
    tick0=1, 
    dtick=1, 
    range=[0.5, 18.],
    title='Streak length',
    showgrid=True, 
    minor=dict(showgrid=True)
)

fig.update_yaxes(
    type='log', 
    title='Probability', 
    range=[-5, 0], 
    showgrid=True, 
    minor=dict(showgrid=True)
)

fig.update_layout(
    hovermode="x",
    margin=dict(l=20, r=20, t=20, b=20),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    width=600, height=300,  
)

# Save to JSON file
with open("docs/loserQ/assets/mock_streak_histogram_loss.json", "w") as f:
    f.write(fig.to_json(pretty=True))
    
fig.show()